# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# load messages dataset
messages = pd.read_csv('messages.csv', delimiter = ',')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
# load categories dataset
categories = pd.read_csv('categories.csv', delimiter = ',')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [4]:
# merge datasets
df = messages.merge(categories, how = 'outer', on = 'id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [5]:
print("Message's shape: {}, Category's shape:{}, After merging: {}".format(messages.shape, categories.shape, df.shape))

Message's shape: (26248, 4), Category's shape:(26248, 2), After merging: (26386, 5)


In [21]:
# check number of missing values in each column
df.isnull().sum().sort_values(ascending = False)/df.shape[0]

original                  0.611688
direct_report             0.000000
search_and_rescue         0.000000
clothing                  0.000000
shelter                   0.000000
food                      0.000000
water                     0.000000
child_alone               0.000000
military                  0.000000
security                  0.000000
medical_products          0.000000
other_weather             0.000000
medical_help              0.000000
aid_related               0.000000
offer                     0.000000
request                   0.000000
related                   0.000000
genre                     0.000000
message                   0.000000
money                     0.000000
missing_people            0.000000
refugees                  0.000000
aid_centers               0.000000
cold                      0.000000
earthquake                0.000000
fire                      0.000000
storm                     0.000000
floods                    0.000000
weather_related     

In [6]:
# create a dataframe of the 36 individual category columns
categories = pd.DataFrame(data = np.zeros((df.shape[0], 36), dtype = int))
#pd.DataFrame(df.categories, columns = ['categories', [i for i in range(36)]], dtype = str)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [7]:
# select the first row of the categories dataframe
row = df.iloc[0]
# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.str.split(';', expand = True).loc['categories'].apply(lambda x : x.split('-')[0])
print(category_colnames)

0                    related
1                    request
2                      offer
3                aid_related
4               medical_help
5           medical_products
6          search_and_rescue
7                   security
8                   military
9                child_alone
10                     water
11                      food
12                   shelter
13                  clothing
14                     money
15            missing_people
16                  refugees
17                     death
18                 other_aid
19    infrastructure_related
20                 transport
21                 buildings
22               electricity
23                     tools
24                 hospitals
25                     shops
26               aid_centers
27      other_infrastructure
28           weather_related
29                    floods
30                     storm
31                      fire
32                earthquake
33                      cold
34            

In [8]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

categories,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [9]:
# convert category values row-wise
for irow, row in enumerate(df.iterrows()):
    # list of categories in each row of df.categories
    cat_list = row[1][4].split(';')
    # only keep the last letter, which is "1" or "0"
    cat_list = [x[-1] for x in cat_list]
    # convert from list to integer array
    cat_array = np.array(cat_list).astype(int)
    # set array values to categories dataframe by row index
    categories.iloc[irow,:] = cat_array
categories.head()

categories,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#categories.tail(10)

In [11]:
#print(df.categories.iloc[26377])

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [10]:
# drop the original categories column from `df`
df.drop(labels = 'categories', axis = 1, inplace = True)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [11]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis = 1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [12]:
# check number of duplicates
df_duplicated = df.duplicated()
n_duplicates = df[df_duplicated == True].shape[0]
print("There are {} duplicates in the dataset:".format(n_duplicates))
df[df_duplicated].head()

There are 170 duplicates in the dataset:


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
164,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
165,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
658,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
659,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
660,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# drop duplicates
df = df[~df_duplicated]

In [14]:
# check number of duplicates
df_duplicated = df.duplicated()
n_duplicates = df[df_duplicated == True].shape[0]
print("There are {} duplicates in the dataset:".format(n_duplicates))

There are 0 duplicates in the dataset:


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLalchemy library.

In [15]:
#engine = create_engine('sqlite:///FigureEight.db') 
#df.to_sql('disaster_response', con=engine, index=False, if_exists = 'replace')
conn = sqlite3.connect('figure_eight.db')
df.to_sql('msg_cat', con = conn, if_exists='replace', index=False)
# commit any changes to the database and close the database
conn.commit()
conn.close()

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

In [ ]:
import sys
import pandas as pd
import numpy as np
import sqlite3

def load_data(messages_filepath, categories_filepath):
    '''
    Usage: read and merge messages and categories csv files into one dataframe objects

    Args:
        messages_filepath (str): filepath and filename for messages file 
        categories_filepath (str): filepath and filename for categories file 
    
    Returns:
        the merged dataframe
    '''
    # read messages file
    print('Reading message dataset.')
    df_messages = pd.read_csv(messages_filepath, delimiter = ',')
    # read categories file
    print('Reading category dataset.')
    df_categories = pd.read_csv(categories_filepath, delimiter = ',')
    # merge datasets
    print('Merging two datasets.')
    df_merge = df_messages.merge(df_categories, how = 'outer', on = 'id')
    return df_merge

def deduplicate(df):
    '''
    Usage: remove all duplicate values in the dataframe
        
    Args:
        df (dataframe): the dataframe you want to perform deduplication
           
    Returns:
        None
    '''
    # check number of duplicates
    df_duplicated = df.duplicated()
    n_duplicates = df[df_duplicated == True].shape[0]
    print("{} duplicates found and removed from the dataset".format(n_duplicates))
    # drop duplicates
    if n_duplicates > 0:
        df = df[~df_duplicated]
    return

def clean_data(df):
    '''
    Usage: merge two datasets of messages categories, and transform into one dataframe with customized format
        
    Args:
        df (dataframe): the dataframe to apply cleaning processes
        
    Returns:
        a cleaned dataframe
    '''
    # create a dataframe of the 36 individual category columns
    categories = pd.DataFrame(data = np.zeros((df.shape[0], 36), dtype = int))
    # split categories into separate category columns
    category_colnames = df.iloc[0] \
                        .str.split(';', expand = True).loc['categories'].apply(lambda x : x.split('-')[0])
    # rename the columns of `categories`
    categories.columns = category_colnames
    print('Parsing category values row-wise.')
    # convert category values row-wise
    for irow, row in enumerate(df.iterrows()):
        # list of categories in each row of df.categories
        cat_list = row[1][4].split(';')
        # only keep the last letter, which is "1" or "0"
        cat_list = [x[-1] for x in cat_list]
        # convert from list to integer array
        cat_array = np.array(cat_list).astype(int)
        # set array values to categories dataframe by row index
        categories.iloc[irow,:] = cat_array
    # drop the original categories column from `df`
    df.drop(labels = 'categories', axis = 1, inplace = True)
    # concatenate the original dataframe with the new `categories` dataframe
    df = pd.concat([df, categories], axis = 1)
    print('Removing duplicated values.')
    # duplication
    deduplicate(df)
    return df
    

def save_data(df, database_filename = 'figure_eight.db', table_name = 'msg_cat'):
    '''
    Usage: load the dataset into the database
        
    Args:
        df (dataframe): the dataframe you want to load
        database_filename (str): the database name
        table_name (str): the table name
            
    Returns:
        None
    '''
    print('Opening connection.')
    conn = sqlite3.connect(database_filename)
    print('Loading data to table "{}" in the database "{}".'.format(table_name, database_filename))
    df.to_sql(table_name, con = conn, if_exists='replace', index=False)
    # commit any changes to the database and close the database
    conn.commit()
    conn.close()
    print('Connection is closed.')
    return None

def read_db(db_name = 'figure_eight.db', tbl_name = 'msg_cat', limit = None):
    '''
    Usage: read data from the database
        
    Args:
        db_name (str): the database name
        tbl_name (str): the table name
        limit (int): the number of returned records, 
                     default - None (return all records)
            
    Returns:
        the dataframe from database queries
    '''
    print('Opening connection.')
    conn = sqlite3.connect(db_name)
    # get a cursor
    cur = conn.cursor()
    cmd = "SELECT * FROM " + tbl_name
    if limit is not None:
        cmd = cmd + ' limit ' + str(limit)
    print('Reading data from table "{}" in the database "{}".'.format(tbl_name, db_name))
    df = pd.read_sql(cmd, con = conn)
    conn.commit()
    conn.close()
    print('Connection is closed.')
    return df


def main():
    if len(sys.argv) == 4:

        messages_filepath, categories_filepath, database_filepath = sys.argv[1:]

        print('Loading data...\n    MESSAGES: {}\n    CATEGORIES: {}'
              .format(messages_filepath, categories_filepath))
        df = load_data(messages_filepath, categories_filepath)

        print('Cleaning data...')
        df = clean_data(df)
        
        print('Saving data...\n    DATABASE: {}'.format(database_filepath))
        save_data(df, database_filepath)
        
        print('Cleaned data saved to database!')
    
    else:
        print('Please provide the filepaths of the messages and categories '\
              'datasets as the first and second argument respectively, as '\
              'well as the filepath of the database to save the cleaned data '\
              'to as the third argument. \n\nExample: python process_data.py '\
              'disaster_messages.csv disaster_categories.csv '\
              'DisasterResponse.db')


if __name__ == '__main__':
    main()

In [18]:
import pandas as pd
import numpy as np
import sqlite3

class ETL_pipeline():
    # load dataset
    def extract(self, msg_file = 'messages.csv', cat_file = 'categories.csv'):
        '''
        Usage: read messages and categories csv files into two dataframe objects
        
        Args:
            msg_file (str): filepath and filename for messages file 
            cat_file (str): filepath and filename for categories file 
    
        Returns:
            two dataframes: df_messages, df_categories
    
        '''
        df_messages = pd.read_csv(msg_file, delimiter = ',')
        df_categories = pd.read_csv(cat_file, delimiter = ',')
        return df_messages, df_categories
    
    def deduplicate(self, df):
        '''
        Usage: remove all duplicate values in the dataframe
        
        Args:
            df (dataframe): the dataframe you want to perform deduplication
           
        Returns:
            None
        '''
        # check number of duplicates
        df_duplicated = df.duplicated()
        n_duplicates = df[df_duplicated == True].shape[0]
        print("{} duplicates found and removed from the dataset".format(n_duplicates))
        # drop duplicates
        if n_duplicates > 0:
            df = df[~df_duplicated]
        return
    
    def transform(self, df_messages, df_categories):
        '''
        Usage: merge two datasets of messages categories, and transform into one dataframe with customized format
        
        Args:
            df_messages (dataframe): the dataframe of message dataset
            df_categories (dataframe): the dataframe of category dataset
            
        Returns:
            a cleaned dataset (dataframe)
        '''
        print('Merging two datasets.')
        # merge datasets
        df_merge = df_messages.merge(df_categories, how = 'outer', on = 'id')
        # create a dataframe of the 36 individual category columns
        categories = pd.DataFrame(data = np.zeros((df_merge.shape[0], 36), dtype = int))
        # split categories into separate category columns
        category_colnames = df_merge.iloc[0] \
                            .str.split(';', expand = True).loc['categories'].apply(lambda x : x.split('-')[0])
        # rename the columns of `categories`
        categories.columns = category_colnames
        print('Parsing category values row-wise.')
        # convert category values row-wise
        for irow, row in enumerate(df_merge.iterrows()):
            # list of categories in each row of df.categories
            cat_list = row[1][4].split(';')
            # only keep the last letter, which is "1" or "0"
            cat_list = [x[-1] for x in cat_list]
            # convert from list to integer array
            cat_array = np.array(cat_list).astype(int)
            # set array values to categories dataframe by row index
            categories.iloc[irow,:] = cat_array
        # drop the original categories column from `df`
        df_merge.drop(labels = 'categories', axis = 1, inplace = True)
        # concatenate the original dataframe with the new `categories` dataframe
        df_merge = pd.concat([df_merge, categories], axis = 1)
        print('Removing duplicated values.')
        # duplication
        self.deduplicate(df_merge)
        return df_merge
    
    def load_db(self, df_results, db_name = 'figure_eight.db', tbl_name = 'msg_cat'):
        '''
        Usage: load the dataset into the database
        
        Args:
            df_results (dataframe): the dataframe you want to load
            db_name (str): the database name
            tbl_name (str): the table name
            
        Returns:
            None
        '''
        print('Opening connection.')
        conn = sqlite3.connect(db_name)
        print('Loading data to table "{}" in the database "{}".'.format(tbl_name, db_name))
        df_results.to_sql(tbl_name, con = conn, if_exists='replace', index=False)
        # commit any changes to the database and close the database
        conn.commit()
        conn.close()
        print('Connection is closed.')
        return None
    
    def read_db(self, db_name = 'figure_eight.db', tbl_name = 'msg_cat', limit = None):
        '''
        Usage: read data from the database
        
        Args:
            db_name (str): the database name
            tbl_name (str): the table name
            limit (int): the number of returned records, 
                         default - None (return all records)
            
        Returns:
            the dataframe from database queries
        '''
        print('Opening connection.')
        conn = sqlite3.connect(db_name)
        # get a cursor
        cur = conn.cursor()
        cmd = "SELECT * FROM " + tbl_name
        if limit is not None:
            cmd = cmd + ' limit ' + str(limit)
        print('Reading data from table "{}" in the database "{}".'.format(tbl_name, db_name))
        df = pd.read_sql(cmd, con = conn)
        conn.commit()
        conn.close()
        print('Connection is closed.')
        return df

In [19]:
pipeline = ETL_pipeline()
df_messages, df_categories = pipeline.extract()
df = pipeline.transform(df_messages, df_categories)
pipeline.load_db(df)
df_db = pipeline.read_db()
df_db.head()

Merging two datasets.
Parsing category values row-wise.
Removing duplicated values.
170 duplicates found and removed from the dataset
Opening connection.
Loading data to table "msg_cat" in the database "figure_eight.db".
Connection is closed.
Opening connection.
Reading data from table "msg_cat" in the database "figure_eight.db".
Connection is closed.


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
